### Usefull other notebooks:
- [EDA with missing data](https://www.kaggle.com/code/calibrator/optiver-2023-eda-wip)
- [Optimization & Visualization with Optuna](https://www.kaggle.com/code/moritzm00/optiver-lgbm-tuning-w-optuna?scriptVersionId=144585564)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import math
import folium # maps
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly import tools
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from eda import load_reduce_memory_dataset

In [2]:
df = load_reduce_memory_dataset("./data/")

Memory usage before optimization is: 1124.11 MB
Memory usage after optimization is: 540.88 MB
Decreased by 51.9%
Memory reduced dataframe is saved successfully


In [3]:
display(df.head(10))
print(df.keys())

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,date_client,temperature_h_mean,dewpoint_h_mean,rain_h_mean,snowfall_h_mean,surface_pressure_h_mean,cloudcover_total_h_mean,cloudcover_low_h_mean,cloudcover_mid_h_mean,cloudcover_high_h_mean,windspeed_10m_h_mean,winddirection_10m_h_mean,shortwave_radiation_h_mean,direct_solar_radiation_h_mean,diffuse_radiation_h_mean,Unnamed: 0_h_mean,hour_h,hours_ahead_f_mean,temperature_f_mean,dewpoint_f_mean,cloudcover_high_f_mean,cloudcover_low_f_mean,cloudcover_mid_f_mean,cloudcover_total_f_mean,10_metre_u_wind_component_f_mean,10_metre_v_wind_component_f_mean,direct_solar_radiation_f_mean,surface_solar_radiation_downwards_f_mean,snowfall_f_mean,total_precipitation_f_mean,Unnamed: 0_f_mean,forecast_date_electricity,euros_per_mwh_electricity,origin_date_electricity,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,origin_date_gas,mean_price_per_mwh_gas,target_2_days_ago,target_3_days_ago,target_4_days_ago,target_5_days_ago,target_6_days_ago,target_7_days_ago,target_8_days_ago,target_9_days_ago,target_10_days_ago,target_11_days_ago,target_12_days_ago,target_13_days_ago,target_14_days_ago,target_15_days_ago
0,0,0,1,0.713000,0,2021-09-01 00:00:00,0,0,0,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.589996,1,2021-09-01 00:00:00,0,1,0,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.313999,1,2021-09-01 00:00:00,0,3,1,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904000,0,2021-09-01 00:00:00,0,4,2,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,3,656.859009,1,2021-09-01 00:00:00,0,5,2,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,1,0,0.000000,0,2021-09-01 00:00:00,0,6,3,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,1,0,59.000000,1,2021-09-01 00:00:00,0,7,3,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,1,1,0.000000,0,2021-09-01 00:00:00,0,8,4,2021-09-01,2021,3,9,35,0,244,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id', 'date',
       'year', 'quarter', 'month', 'week', 'hour', 'day_of_year',
       'day_of_month', 'day_of_week', 'eic_count_client',
       'installed_capacity_client', 'date_client', 'temperature_h_mean',
       'dewpoint_h_mean', 'rain_h_mean', 'snowfall_h_mean',
       'surface_pressure_h_mean', 'cloudcover_total_h_mean',
       'cloudcover_low_h_mean', 'cloudcover_mid_h_mean',
       'cloudcover_high_h_mean', 'windspeed_10m_h_mean',
       'winddirection_10m_h_mean', 'shortwave_radiation_h_mean',
       'direct_solar_radiation_h_mean', 'diffuse_radiation_h_mean',
       'Unnamed: 0_h_mean', 'hour_h', 'hours_ahead_f_mean',
       'temperature_f_mean', 'dewpoint_f_mean', 'cloudcover_high_f_mean',
       'cloudcover_low_f_mean', 'cloudcover_mid_f_mean',
       'cloudcover_total_f_mean', '10_metre_u_wind_component_f_mean',
       '10_metre_v_win

In [6]:
print(df["latitude"])

KeyError: 'latitude'

In [ ]:
print(f"--- Duplicated_rows --- ")
print(df.duplicated().sum())
print(f"--- Nans rows --- ")
df.isnull().sum()/len(df)

# check df properties
def analyze_df(df : pd.DataFrame)->pd.DataFrame:
    res = pd.DataFrame({
        "is_unique": df.nunique() == len(df),
        "unique": df.nunique(),
        "with_nan":df.isna().any(),
        "percent_nan":round((df.isnull().sum()/len(df))*100,4),
        "dtype":df.dtypes
    })
    return res
analyze_df(df=df)

--- 
# Geography

In [ ]:
map_ = folium.Map(location=[58.595272, 25.013607], zoom_start=7)
counties_locations = {df["county"].unique():(x,y) for (x,y) in zip(df["county"])}
for county, coords in counties_locations.items():
    folium.CircleMarker(
        location=[coords[0], coords[1]], radius=5, color='cornflowerblue', fill=True
    ).add_child(folium.Popup(county)).add_to(map_)

for _, loc in weather_locations.iterrows():
    folium.CircleMarker(
        location=[loc["latitude"], loc["longitude"]], radius=1, color='orange'
    ).add_to(map_)

map_

---
# EDA with plotly

In [22]:

def _plot_target(df:pd.DataFrame):
    """ plot the target using plotly """
    # fig = px.area(x=df["date"][::100],y=df["is_consumption"][::100])
    #fig = px.area(x=df["date"][::100],y=df["target"][::100],log_y=True)
    #fig.add_trace(go.Scatter(x=df["date"][::100],y=df["target"][::100]*df["is_consumption"][::100],opacity=.5))
    fig = px.scatter(x=df["date"][::100],y=df["target"][::100]*df["is_consumption"][::100],log_y=True)
    fig.add_hline(y=df["target"][::100].mean())#,
                  # line_dash='dot',annotation="Average target",
                  # color="gray")#,annotation_position="bottom right")
    #fig.add_traces(line_color="red")
    fig.update_layout(xaxis_title="Date",yaxis_title="Consumption")
    fig.show()
_plot_target(df=df)

In [12]:
def _show_targets(df: pd.DataFrame, n_uids: int = 6) -> None:
    """Randomly sample and plot target sequences."""
    n_cols_per_row = 3
    colors = {0: "rgba(8, 209, 59, 0.5)", 1: "rgba(245, 10, 10, 0.7)"}
    sampled_ids = np.random.choice(df["prediction_unit_id"].unique(), size=n_uids, replace=False)

    fig = make_subplots(rows=math.ceil(n_uids / n_cols_per_row), cols=n_cols_per_row)
    for i, uid in enumerate(sampled_ids):
        train_uid = (
            df
            .filter((df["prediction_unit_id"] == uid))
            .sort("datetime")
        )
        for is_cons in [0, 1]:
            train_uid_ = train_uid.filter(df["is_consumption"] == is_cons)
            row, col = i//n_cols_per_row + 1, i%n_cols_per_row + 1
            fig.add_trace(
                go.Scatter(
                    x=train_uid_["datetime"],
                    y=train_uid_["target"],
                    name="Consumption" if is_cons else "Production",
                    line={"color": colors[is_cons]},
                    legendgroup=i,
                    legendgrouptitle_text=f"Plot ({row}, {col}) - UID {uid}"
                ),
                row=row,
                col=col
            )

    fig.update_layout(
        title_text=f"Sampled Electricity Sequences",
        margin=dict(l=12, r=12, t=50, b=50)
    )
    fig.show()
_show_targets(df=df, n_uids = 6)

AttributeError: 'DataFrame' object has no attribute 'sort'

In [ ]:
df = load_reduce_memory_dataset("./data/")
fig = px.area(df,
              x="datetime",
              y="target",
              title='1. Daily Consumption Analysis')
fig.add_hline(y=df["target"].mean(), line_dash="dot",
              annotation_text="Average Consumption",
              annotation_position="bottom right")
fig.update_traces(line_color='#FA163F')
fig.update_layout(xaxis_title="Date",
                  yaxis_title="Consumption")
fig.show()